# Imports

In [2]:
import os
import time
import json
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from scipy import stats
from scipy.io.arff import loadarff
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.preprocessing import LabelBinarizer, StandardScaler
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from imblearn.over_sampling import SMOTE

In [3]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

# Helper functions

In [4]:
# função para criar os diretórios
def creat_dir(model_name):
    if not 'log_metrics' in os.listdir():
        os.mkdir('log_metrics')
    if not 'imgs' in os.listdir():
        os.mkdir('imgs')
    if not model_name in os.listdir('./imgs'):
        os.mkdir('./imgs/'+model_name)

In [5]:
# função para adicionar as métricas de um fold em um dicionário
def dic_par_metrics(y_test, y_onehot_test, y_pred, y_proba, grid_search_cv):
    accuracy = metrics.accuracy_score(y_test, y_pred)
    balanced_accuracy = metrics.balanced_accuracy_score(y_test, y_pred)
    precision_micro = metrics.precision_score(y_test, y_pred, average='micro')
    precision_macro = metrics.precision_score(y_test, y_pred, average='macro')
    precision_weighted = metrics.precision_score(y_test, y_pred, average='weighted')
    recall_micro = metrics.recall_score(y_test, y_pred, average='micro')
    recall_macro = metrics.recall_score(y_test, y_pred, average='macro')
    recall_weighted = metrics.recall_score(y_test, y_pred, average='weighted')
    f1_micro = metrics.f1_score(y_test, y_pred, average='micro')
    f1_macro = metrics.f1_score(y_test, y_pred, average='macro')
    f1_weighted = metrics.f1_score(y_test, y_pred, average='weighted')
    roc_auc_ovr_micro = metrics.roc_auc_score(y_onehot_test, y_proba, multi_class='ovr', average='micro')
    roc_auc_ovo_micro = metrics.roc_auc_score(y_onehot_test, y_proba, multi_class='ovo', average='micro')
    roc_auc_ovr_macro = metrics.roc_auc_score(y_onehot_test, y_proba, multi_class='ovr', average='macro')
    roc_auc_ovo_macro = metrics.roc_auc_score(y_onehot_test, y_proba, multi_class='ovo', average='macro')
    roc_auc_ovr_weighted = metrics.roc_auc_score(y_onehot_test, y_proba, multi_class='ovr', average='weighted')
    roc_auc_ovo_weighted = metrics.roc_auc_score(y_onehot_test, y_proba, multi_class='ovo', average='weighted')
    classification_report = metrics.classification_report(y_test, y_pred, output_dict=True)
    dicMetricas = {
        "parameters": grid_search_cv.best_params_,
        "metrics":{
            "accuracy": accuracy,
            "balanced_accuracy": balanced_accuracy,
            "precision_micro": precision_micro,
            "precision_macro": precision_macro,
            "precision_weighted": precision_weighted,
            "recall_micro": recall_micro,
            "recall_macro": recall_macro,
            "recall_weighted": recall_weighted,
            "f1_micro": f1_micro,
            "f1_macro": f1_macro,
            "f1_weighted": f1_weighted,
            "roc_auc_ovr_micro": roc_auc_ovr_micro,
            "roc_auc_ovo_micro": roc_auc_ovo_micro,
            "roc_auc_ovr_macro": roc_auc_ovr_macro,
            "roc_auc_ovo_macro": roc_auc_ovo_macro,
            "roc_auc_ovr_weighted": roc_auc_ovr_weighted,
            "roc_auc_ovo_weighted": roc_auc_ovo_weighted
        },
        "classification_report": classification_report
    }
    return dicMetricas

In [6]:
# função para calcular a média e o desvio padrão das métricas de cada fold, além de plotar os boxplots
def calc_mean_std(dic_json, model_name):
    key = list(dic_json.keys())[0]
    dic_mean = {}
    dic_std = {}
    for j in dic_json[key]['metrics'].keys():
        metrics_list = []
        for i in dic_json.keys():
            metrics_list.append(dic_json[i]['metrics'][j])
        mean = np.mean(metrics_list)
        std = np.std(metrics_list)
        dic_mean.update({j: mean})
        dic_std.update({j: std})
        plt.close('all')
        plt.boxplot(metrics_list, labels=[model_name])
        plt.ylabel(j)
        plt.savefig('./imgs/'+model_name+'/boxplot_'+j+'.png')
        plt.close('all')
    dic_json.update({"mean": dic_mean})
    dic_json.update({"std": dic_std})
    return dic_json

In [7]:
def plot_matriz_confusao_one_vs_one(y_test, y_pred, model_name, fold_i, flag_normalizado="true"):
    """Plota a matriz de confusao comparando cada classe entre si, mostrando as predicoes contra as
    classes verdadeiras.

    Args:
        y_test: serie com os valores verdadeiros
        y_pred: serie com os valores preditos
        flag_normalizado (optional): flag indicando se os valores da matriz de confusao devem ser
        normalizados. Se devem ser normalizados pel. Defaults to None.
    """
    class_names = np.unique(y_pred)

    fig, ax = plt.subplots(figsize=(12, 10))
    cm_plot = metrics.ConfusionMatrixDisplay.from_predictions(
        y_test, y_pred, cmap="YlOrRd", normalize=flag_normalizado, ax=ax)
    ax.set_xticklabels(class_names, rotation=45, ha="right")
    ax.set_title(f"Matriz de Confusao - Modelo {model_name} - Fold {fold_i}")

    cm_plot.figure_.savefig(
        f"imgs/{model_name}/cm_normalizado_{flag_normalizado}_{model_name}_fold_{fold_i}.png", 
        dpi=300
    )
    plt.close()
    
    return cm_plot

In [8]:
def inicializa_roc_one_class_vs_rest_kfold(y_onehot_test, y_pred_score, i, class_id, ax, tprs, aucs, mean_fpr):
    viz = metrics.RocCurveDisplay.from_predictions(
        y_onehot_test[:, class_id],
        y_pred_score[:, class_id],
        name=f"ROC OvR Class_{class_id+1} fold {i}",
        alpha=0.3,
        lw=1,
        ax=ax,
        plot_chance_level=(i==9),
    )

    interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)
    interp_tpr[0] = 0.0
    tprs.append(interp_tpr)
    aucs.append(viz.roc_auc)

In [9]:
def gera_roc_one_class_vs_rest_kfold(tprs, aucs, mean_fpr, plot, model_name, class_id):
    fig = plot[0]
    ax = plot[1]
    
    mean_tpr = np.mean(tprs, axis=0)
    mean_tpr[-1] = 1.0
    mean_auc = metrics.auc(mean_fpr, mean_tpr)
    std_auc = np.std(aucs)
    ax.plot(
        mean_fpr,
        mean_tpr,
        color="b",
        label=r"Mean ROC (AUC = %0.2f $\pm$ %0.2f)" % (mean_auc, std_auc),
        lw=2,
        alpha=0.8,
    )
    std_tpr = np.std(tprs, axis=0)
    tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
    tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
    ax.fill_between(
        mean_fpr,
        tprs_lower,
        tprs_upper,
        color="grey",
        alpha=0.2,
        label=r"$\pm$ 1 std. dev.",
    )
    ax.set(
        xlim=[-0.05, 1.05],
        ylim=[-0.05, 1.05],
        xlabel="False Positive Rate",
        ylabel="True Positive Rate",
        title=f"Curva ROC media com desvio padrao\nROC OvR Class_{class_id+1}",
    )
    ax.axis("square")
    ax.legend(loc="lower right")

    fig.savefig(f"imgs/{model_name}/roc_{model_name}_class_{class_id+1}.png", dpi=300)
    plt.close()

In [10]:
def inicializa_roc_micro_average_kfold(y_onehot_test, y_pred_score, i, ax, tprs, aucs, mean_fpr):
    viz = metrics.RocCurveDisplay.from_predictions(
        y_onehot_test.ravel(),
        y_pred_score.ravel(),
        name=f"micro-average OvR fold {i}",
        alpha=0.3,
        lw=1,
        ax=ax,
        plot_chance_level=(i==9)
    )

    interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)
    interp_tpr[0] = 0.0
    tprs.append(interp_tpr)
    aucs.append(viz.roc_auc)

In [11]:
def gera_roc_micro_average_kfold(tprs, aucs, mean_fpr, fig, ax, model_name):
    mean_tpr = np.mean(tprs, axis=0)
    mean_tpr[-1] = 1.0
    mean_auc = metrics.auc(mean_fpr, mean_tpr)
    std_auc = np.std(aucs)
    ax.plot(
        mean_fpr,
        mean_tpr,
        color="b",
        label=r"Mean ROC (AUC = %0.2f $\pm$ %0.2f)" % (mean_auc, std_auc),
        lw=2,
        alpha=0.8,
    )
    std_tpr = np.std(tprs, axis=0)
    tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
    tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
    ax.fill_between(
        mean_fpr,
        tprs_lower,
        tprs_upper,
        color="grey",
        alpha=0.2,
        label=r"$\pm$ 1 std. dev.",
    )
    ax.set(
        xlim=[-0.05, 1.05],
        ylim=[-0.05, 1.05],
        xlabel="False Positive Rate",
        ylabel="True Positive Rate",
        title=f"Curva ROC media com desvio padrao\nROC micro-average",
    )
    ax.axis("square")
    ax.legend(loc="lower right")

    fig.savefig(f"imgs/{model_name}/roc_micro_average_{model_name}.png", dpi=300)
    plt.close()

In [12]:
def stratified_k_fold_grid_search_cv(model, params:dict, X, y, model_name):
    """Realiza o treino e validacao do modelo utilizando StratifiedKFold com GridSearchCV para
    busca dos melhores hiperparametros (tendo assim um nested cross-validation).

    Args:
        model: modelo que se deseja treinar e validar
        params (dict): dicionario com os parametros e seus respectivos valores possiveis para o 
        grid search
        X: features do modelo
        y: coluna de target do modelo
    """
    startTime = time.time()
    # criando os diretórios necessários para salvar as métricas e os plots
    creat_dir(model_name)

    print(f"Modelo: {model_name}")
    dic_json = {}

    n_splits_k_fold = 10
    n_splits_grid_search = 5

    matriz_tprs = [[] for i in range(9)]
    matriz_aucs = [[] for i in range(9)]
    mean_fpr = np.linspace(0, 1, 100)
    lista_plots = [plt.subplots(figsize=(6, 6)) for i in range(9)]

    tprs_micro_average = []
    aucs_micro_average = []
    fig_micro_average, ax_micro_average = plt.subplots(figsize=(6, 6))

    skf = StratifiedKFold(n_splits=n_splits_k_fold, random_state=4, shuffle=True)
    for i, (train_index, test_index) in enumerate(skf.split(X, y)):
        print(f"## INICIO FOLD {i} ##")
        # separa os dados de treino e teste
        X_train = X.iloc[train_index]
        X_test = X.iloc[test_index]
        y_train = y.iloc[train_index]
        y_test = y.iloc[test_index]
        # normaliza os dados
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
        # binariza a coluna target para uso no plot roc e metrica roc_auc_score
        label_binarizer = LabelBinarizer().fit(y_train)
        y_onehot_test = label_binarizer.transform(y_test)
        # prepara o y_train
        y_train = y_train.values.ravel()
        # inicializa e roda o grid search
        grid_search_cv = GridSearchCV(
            estimator=model, param_grid=params, scoring="f1_weighted", cv=n_splits_grid_search, 
            verbose=3)
        grid_search_cv.fit(X_train, y_train)
        # com os melhores parametros encontrados, realiza a predicao no fold de teste e calcula a 
        # metrica de avaliacao
        y_pred = grid_search_cv.predict(X_test)
        # verifica se o modelo em execucao eh o perceptron, ja que o mesmo nao possui a funcao 
        # predict_proba
        if model_name != 'Perceptron':
            y_proba = grid_search_cv.predict_proba(X_test)
        else:
            y_proba = grid_search_cv.decision_function(X_test)
        # armazenando as metricas e os parametros em um dicionario
        dic_fold = dic_par_metrics(y_test, y_onehot_test, y_pred, y_proba, grid_search_cv)
        dic_json.update({"fold "+str(i): dic_fold})
        # gera a matriz de confusao com dados normalizados e nao normalizados e salva ambas versoes
        plot_matriz_confusao_one_vs_one(y_test, y_pred, model_name, i, flag_normalizado="true")
        plot_matriz_confusao_one_vs_one(y_test, y_pred, model_name, i, flag_normalizado=None)
        # inicializa a curva roc para cada classe neste fold
        for class_id, plot, tprs, aucs in zip(range(9), lista_plots, matriz_tprs, matriz_aucs):
            inicializa_roc_one_class_vs_rest_kfold(
                y_onehot_test, y_proba, i, class_id, plot[1], tprs, aucs, mean_fpr
            )
        # inicializa a curva roc micro average neste fold
        inicializa_roc_micro_average_kfold(
            y_onehot_test, y_proba, i, ax_micro_average, tprs_micro_average, 
            aucs_micro_average, mean_fpr
        )
        print(f"## FINAL FOLD {i} ##\n", dic_fold)

    endTime = time.time()
    # inserindo a média dos folds no dicionário
    dic_json = calc_mean_std(dic_json, model_name)
    # inserindo o tempo de execução
    dic_json.update({"time": endTime-startTime})
    # salvando o dicionário no formato json
    objOpen = open(f'./log_metrics/{model_name}.json', 'w')
    objOpen.write(json.dumps(dic_json, indent=4))
    objOpen.close()

    # gera o plot da curva roc para cada classe, juntando todos os folds e realizando uma media
    for class_id, plot, tprs, aucs in zip(range(9), lista_plots, matriz_tprs, matriz_aucs):
        gera_roc_one_class_vs_rest_kfold(tprs, aucs, mean_fpr, plot, model_name, class_id)
    # gera o plot da curva roc micro average, juntando todos os folds e realizando uma media 
    gera_roc_micro_average_kfold(
        tprs_micro_average, aucs_micro_average, mean_fpr, fig_micro_average, ax_micro_average, 
        model_name
    )

# Carrega dados

In [13]:
raw_data = loadarff('data/dataset.arff')
df_data = pd.DataFrame(raw_data[0])
df_data.head()

,id,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,feat_10,feat_11,feat_12,feat_13,feat_14,feat_15,feat_16,feat_17,feat_18,feat_19,feat_20,feat_21,feat_22,feat_23,feat_24,feat_25,feat_26,feat_27,feat_28,feat_29,feat_30,feat_31,feat_32,feat_33,feat_34,feat_35,feat_36,feat_37,feat_38,feat_39,feat_40,feat_41,feat_42,feat_43,feat_44,feat_45,feat_46,feat_47,feat_48,feat_49,feat_50,feat_51,feat_52,feat_53,feat_54,feat_55,feat_56,feat_57,feat_58,feat_59,feat_60,feat_61,feat_62,feat_63,feat_64,feat_65,feat_66,feat_67,feat_68,feat_69,feat_70,feat_71,feat_72,feat_73,feat_74,feat_75,feat_76,feat_77,feat_78,feat_79,feat_80,feat_81,feat_82,feat_83,feat_84,feat_85,feat_86,feat_87,feat_88,feat_89,feat_90,feat_91,feat_92,feat_93,target
0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,4.0,1.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,11.0,0.0,1.0,1.0,0.0,1.0,0.0,7.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,b'Class_1'
1,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,b'Class_1'
2,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,b'Class_1'
3,4.0,1.0,0.0,0.0,1.0,6.0,1.0,5.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,2.0,2.0,0.0,0.0,0.0,58.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,1.0,2.0,1.0,3.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,5.0,0.0,0.0,4.0,0.0,0.0,2.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,2.0,2.0,0.0,22.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,b'Class_1'
4,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,b'Class_1'


In [14]:
# decodifica string de target
df_data["target"] = df_data["target"].str.decode("utf-8")

In [15]:
df_data.sample(5)

,id,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,feat_10,feat_11,feat_12,feat_13,feat_14,feat_15,feat_16,feat_17,feat_18,feat_19,feat_20,feat_21,feat_22,feat_23,feat_24,feat_25,feat_26,feat_27,feat_28,feat_29,feat_30,feat_31,feat_32,feat_33,feat_34,feat_35,feat_36,feat_37,feat_38,feat_39,feat_40,feat_41,feat_42,feat_43,feat_44,feat_45,feat_46,feat_47,feat_48,feat_49,feat_50,feat_51,feat_52,feat_53,feat_54,feat_55,feat_56,feat_57,feat_58,feat_59,feat_60,feat_61,feat_62,feat_63,feat_64,feat_65,feat_66,feat_67,feat_68,feat_69,feat_70,feat_71,feat_72,feat_73,feat_74,feat_75,feat_76,feat_77,feat_78,feat_79,feat_80,feat_81,feat_82,feat_83,feat_84,feat_85,feat_86,feat_87,feat_88,feat_89,feat_90,feat_91,feat_92,feat_93,target
33971,33972.0,0.0,0.0,10.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,0.0,1.0,0.0,0.0,3.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,10.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,19.0,2.0,0.0,0.0,0.0,0.0,0.0,6.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,6.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,0.0,Class_6
27692,27693.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,4.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,Class_4
53548,53549.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Class_8
51423,51424.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,8.0,1.0,0.0,1.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,2.0,5.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,Class_8
30912,30913.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,6.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Class_5


In [16]:
# verificando o número de linhas e colunas do dataset
print("Shape dataset:", df_data.shape)

Shape dataset: (61878, 95)


In [17]:
# verificando o menor valor das colunas de características
print("Valor mínimo das features:", df_data.iloc[:,1:-1].min().min())

Valor mínimo das features: 0.0


In [18]:
# verificando o maior valor das colunas de características
print("Valor máximo das features:", df_data.iloc[:,1:-1].max().max())

Valor máximo das features: 352.0


In [19]:
# verificando o número de instâncias para cada classe
print('Número de Instâncias por Classe:\n', df_data.groupby('target').count()['id'])

Número de Instâncias por Classe:
 target
Class_1     1929
Class_2    16122
Class_3     8004
Class_4     2691
Class_5     2739
Class_6    14135
Class_7     2839
Class_8     8464
Class_9     4955
Name: id, dtype: int64


In [20]:
# verificando o percentual de instâncias para cada classe
print('Percentual de Instâncias por Classe:\n', (df_data.groupby('target').count()['id']/df_data.shape[0])*100)

Percentual de Instâncias por Classe:
 target
Class_1     3.117425
Class_2    26.054494
Class_3    12.935130
Class_4     4.348880
Class_5     4.426452
Class_6    22.843337
Class_7     4.588060
Class_8    13.678529
Class_9     8.007693
Name: id, dtype: float64


In [21]:
# verificando se existem valores NaN
print('\nNúmero de Valores NaN:', df_data.isnull().sum().sum())


Número de Valores NaN: 0


In [22]:
df_data.describe()

,id,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,feat_10,feat_11,feat_12,feat_13,feat_14,feat_15,feat_16,feat_17,feat_18,feat_19,feat_20,feat_21,feat_22,feat_23,feat_24,feat_25,feat_26,feat_27,feat_28,feat_29,feat_30,feat_31,feat_32,feat_33,feat_34,feat_35,feat_36,feat_37,feat_38,feat_39,feat_40,feat_41,feat_42,feat_43,feat_44,feat_45,feat_46,feat_47,feat_48,feat_49,feat_50,feat_51,feat_52,feat_53,feat_54,feat_55,feat_56,feat_57,feat_58,feat_59,feat_60,feat_61,feat_62,feat_63,feat_64,feat_65,feat_66,feat_67,feat_68,feat_69,feat_70,feat_71,feat_72,feat_73,feat_74,feat_75,feat_76,feat_77,feat_78,feat_79,feat_80,feat_81,feat_82,feat_83,feat_84,feat_85,feat_86,feat_87,feat_88,feat_89,feat_90,feat_91,feat_92,feat_93
count,61878.000000,61878.00000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000
mean,30939.500000,0.38668,0.263066,0.901467,0.779081,0.071043,0.025696,0.193704,0.662433,1.011296,0.263906,1.252869,0.140874,0.480979,1.696693,1.284398,1.413459,0.366108,0.575423,0.551699,0.471525,0.204014,0.729969,0.142522,2.643880,1.534520,0.563108,0.696613,0.238970,0.275768,0.150312,0.148680,1.043796,0.696516,0.946411,0.666263,0.709089,0.263632,0.582129,0.485585,1.653059,0.303468,0.698019,0.451146,0.560829,0.238130,0.641375,0.249669,1.584893,0.348314,0.324283,0.053298,0.213485,0.442063,2.072465,0.323120,0.303775,0.309108,0.697970,0.388603,1.029930,0.239746,1.187563,0.168590,1.256796,0.222228,0.571706,2.897653,0.392902,0.811128,0.892789,0.319290,0.858722,0.591050,0.579851,0.726817,0.748457,0.124196,0.366415,0.300446,0.698067,0.078461,0.187983,0.496719,0.070752,0.532306,1.128576,0.393549,0.874915,0.457772,0.812421,0.264941,0.380119,0.126135
std,17862.784315,1.52533,1.252073,2.934818,2.788005,0.438902,0.215333,1.030102,2.255770,3.474822,1.083340,3.042333,0.567089,2.014697,3.163212,3.862236,2.226163,1.477436,1.335985,4.636145,1.438727,0.696050,1.446220,0.782979,4.629015,2.332994,1.710305,2.873222,0.828112,1.901294,1.640880,0.897354,2.416849,1.310202,3.368622,3.197965,2.555119,0.756934,1.602579,3.298315,3.299798,1.085672,1.961189,1.706013,1.346090,2.587131,2.348359,1.446203,2.577071,1.369380,1.720470,0.513820,1.044788,2.006485,4.113319,0.998743,1.925806,1.082148,3.983722,2.577693,3.028469,1.017553,2.666742,0.946158,3.402080,0.783052,1.361874,4.974322,1.761054,4.111091,1.941368,1.162443,2.411646,5.783233,3.757822,3.200095,2.920038,0.906621,2.778317,1.285569,2.245671,0.461244,0.836269,2.434921,1.151460,1.900438,2.681554,1.575455,2.115466,1.527385,4.597804,2.045646,0.982385,1.201720
min,1.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,

# Modelos

In [23]:
X = df_data.drop(columns=["id", "target"])
y = df_data[["target"]]

In [27]:
y.value_counts()

target 
Class_2    16122
Class_6    14135
Class_8     8464
Class_3     8004
Class_9     4955
Class_7     2839
Class_5     2739
Class_4     2691
Class_1     1929
Name: count, dtype: int64

In [28]:
smote = SMOTE(random_state=4)
X_smote, y_smote = smote.fit_resample(X, y)

In [29]:
y_smote.value_counts()

target 
Class_1    16122
Class_2    16122
Class_3    16122
Class_4    16122
Class_5    16122
Class_6    16122
Class_7    16122
Class_8    16122
Class_9    16122
Name: count, dtype: int64

## KNN

In [ ]:
# define os parametros e seus respectivos valores a serem testados no grid search
params = {
    "n_neighbors": [5, 10, 15]
}
# define o modelo
model = KNeighborsClassifier(n_jobs=1)
# chama a funcao que roda o stratified k fold e valida o modelo realizando a busca por hiper 
# parametros com grid search cv, fazendo assim um nested cross-validation
stratified_k_fold_grid_search_cv(model, params, X, y, "KNN")

## DecisionTree

In [ ]:
# define os parametros e seus respectivos valores a serem testados no grid search
params = {
    "criterion": ["gini", "entropy", "log_loss"],
    "class_weight": ["balanced", None],
    "max_depth": [None, 50, 100, 200, 500, 1000]
}
# define o modelo
model = DecisionTreeClassifier(random_state=4)
# chama a funcao que roda o stratified k fold e valida o modelo realizando a busca por hiper 
# parametros com grid search cv, fazendo assim um nested cross-validation
stratified_k_fold_grid_search_cv(model, params, X, y, "DecisionTree")

## Random Forest

In [ ]:
# define os parametros e seus respectivos valores a serem testados no grid search
params = {
    "n_estimators": [100, 300, 500],
    "max_depth": [None, 400, 800],
    "class_weight": ["balanced_subsample", None]
}

# define o modelo
model = RandomForestClassifier(random_state=4)
# chama a funcao que roda o stratified k fold e valida o modelo realizando a busca por hiper 
# parametros com grid search cv, fazendo assim um nested cross-validation
stratified_k_fold_grid_search_cv(model, params, X, y, "RandomForest")

## Naive Bayes (Gaussian)

In [ ]:
# define os parametros e seus respectivos valores a serem testados no grid search
params = {
}

# define o modelo
model = GaussianNB()
# chama a funcao que roda o stratified k fold e valida o modelo realizando a busca por hiper 
# parametros com grid search cv, fazendo assim um nested cross-validation
stratified_k_fold_grid_search_cv(model, params, X, y, "NaiveBayes")

## Logistic Regression

In [ ]:
# define os parametros e seus respectivos valores a serem testados no grid search
params = {
    "penalty": [None, "l2"],
    "class_weight": ["balanced", None],
    "max_iter": [100, 300, 500, 700, 1000]
}

# define o modelo
model = LogisticRegression(random_state=4)
# chama a funcao que roda o stratified k fold e valida o modelo realizando a busca por hiper 
# parametros com grid search cv, fazendo assim um nested cross-validation
stratified_k_fold_grid_search_cv(model, params, X, y, "LogisticRegression")

## Perceptron

In [ ]:
# define os parametros e seus respectivos valores a serem testados no grid search
params = {
    "penalty": [None, "l2"],
    "class_weight": ["balanced", None],
    "max_iter": [100, 300, 500, 700, 1000]
}

# define o modelo
model = Perceptron(random_state=4)
# chama a funcao que roda o stratified k fold e valida o modelo realizando a busca por hiper 
# parametros com grid search cv, fazendo assim um nested cross-validation
stratified_k_fold_grid_search_cv(model, params, X, y, "Perceptron")

## MLP

In [ ]:
# define os parametros e seus respectivos valores a serem testados no grid search
params = {
    "learning_rate_init": [0.001, 0.01],
    "max_iter": [200, 500, 700],
    "hidden_layer_sizes": [20, 50, 100]
}

# define o modelo
model = MLPClassifier(random_state=4)
# chama a funcao que roda o stratified k fold e valida o modelo realizando a busca por hiper 
# parametros com grid search cv, fazendo assim um nested cross-validation
stratified_k_fold_grid_search_cv(model, params, X, y, "MLPClassifier")

## SVM

In [ ]:
# define os parametros e seus respectivos valores a serem testados no grid search
params = {
    "C": [1],
    "kernel": ["rbf", "linear"]
}

# define o modelo
model = SVC(random_state=4, probability=True)
# chama a funcao que roda o stratified k fold e valida o modelo realizando a busca por hiper 
# parametros com grid search cv, fazendo assim um nested cross-validation
stratified_k_fold_grid_search_cv(model, params, X, y, "SVM")

# Teste de Hipótese

Vamos avaliar se a média dos resultados obtidos com o modelo Random Forest é maior que a média dos resultados obtidos com o modelo MLP. Iremos realizar um teste de hipótese para checar se, estatisticamente, podemos constatar essa afirmação.

In [ ]:
with open("log_metrics/RandomForest.json") as json_file:
    json_rf = json.load(json_file)

lista_resultados_rf = []
for key in list(json_rf.keys()):
    if (key != "mean") & (key != "std") & (key != "time"):
        lista_resultados_rf.append(json_rf[key]["metrics"]["f1_weighted"])

with open("log_metrics/MLPClassifier.json") as json_file:
    json_mlp = json.load(json_file)

lista_resultados_mlp = []
for key in list(json_mlp.keys()):
    if (key != "mean") & (key != "std") & (key != "time"):
        lista_resultados_mlp.append(json_mlp[key]["metrics"]["f1_weighted"])

df_resultados = pd.DataFrame({"rf": lista_resultados_rf, "mlp": lista_resultados_mlp})

In [ ]:
fig, ax = plt.subplots(1, 2)
sns.set(rc={'figure.figsize':(10,5)})

sns.histplot(df_resultados, x="rf", ax=ax[0]).set_title("Distribuicao Random Forest")
sns.histplot(df_resultados, x="mlp", ax=ax[1]).set_title("Distribuicao MLP");

In [ ]:
print(
    f"Teste normalidade random forest: {stats.shapiro(df_resultados['rf'])}\n"
    f"Teste normalidade mlp: {stats.shapiro(df_resultados['mlp'])}"
)

In [ ]:
stats.mannwhitneyu(df_resultados["rf"], df_resultados["mlp"], alternative="greater")

In [ ]:
print(
    f"Variancia random forest: {np.var(df_resultados['rf'])}\n"
    f"Variancia mlp: {np.var(df_resultados['mlp'])}"
)

In [ ]:
stats.ttest_ind(a=df_resultados["rf"], b=df_resultados["mlp"], equal_var=True, alternative="greater")